In [0]:
%run ./10-kafka-to-bronze

In [0]:
# dbutils.fs.rm("/FileStore/data_spark_streaming/checkpoint/invoices_kafka_bz", True)

True

In [0]:
class kafkaToBronzeTestSuite():
    def __init__(self):
        self.base_data_dir = "/FileStore/data_spark_streaming"

    def cleanTests(self):
        print(f"Starting Cleanup...", end='')
        spark.sql("drop table if exists invoices_bz")
        dbutils.fs.rm("/user/hive/warehouse/invoices_bz", True)
        dbutils.fs.rm(f"{self.base_data_dir}/checkpoint/invoices_bz", True)
        print("Done")

    def assertResult(self, expected_count):
        print(f"\tStarting validation...", end='')
        actual_count = spark.sql("select count(*) from invoices_bz").collect()[0][0]
        assert expected_count == actual_count, f"Test failed! actual count is {actual_count}"
        print("Done")

    def waitForMicroBatch(self, sleep=30):
        import time
        print(f"\tWaiting for {sleep} seconds...", end='')
        time.sleep(sleep)
        print("Done.")    

    def runTests(self):
        self.cleanTests()
        bzStream = Bronze()        

        print("Testing Scenario - Start from beginning on a new checkpoint...") 
        bzQuery = bzStream.process()
        self.waitForMicroBatch() 
        bzQuery.stop()       
        self.assertResult(30)
        print("Validation passed.\n")        

        print("Testing Scenarion - Restart from where it stopped on the same checkpoint...")
        bzQuery = bzStream.process()
        self.waitForMicroBatch()
        bzQuery.stop()
        self.assertResult(30)
        print("Validation passed.\n") 

        print("Testing Scenario - Start from 1697945539000 on a new checkpoint...") 
        dbutils.fs.rm(f"{self.base_data_dir}/checkpoint/invoices_bz", True)
        bzQuery = bzStream.process(1697945539000)
        self.waitForMicroBatch()
        bzQuery.stop()
        self.assertResult(40)
        print("Validation passed.\n") 

In [0]:
ts = kafkaToBronzeTestSuite()
ts.runTests()

Starting Cleanup...Done
Testing Scenario - Start from beginning on a new checkpoint...
Starting Bronze Streaming Job...
Done
	Waiting for 30 seconds...Done.


---------------------------------------------------------------------------
Py4JJavaError                             Traceback (most recent call last)
File <command-990638186221114>, line 2
      1 ts = kafkaToBronzeTestSuite()
----> 2 ts.runTests()

File <command-990638186221112>, line 31, in kafkaToBronzeTestSuite.runTests(self)
     29 bzQuery = bzStream.process()
     30 self.waitForMicroBatch() 
---> 31 bzQuery.stop()       
     32 self.assertResult(30)
     33 print("Validation passed.\n")        

File /databricks/spark/python/pyspark/sql/streaming/query.py:372, in StreamingQuery.stop(self)
    349 def stop(self) -> None:
    350     """
    351     Stop this streaming query.
    352 
   (...)
    370     False
    371     """
--> 372     self._jsq.stop()

File /databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py:1355, in JavaMember.__call__(self, *args)
   1349 command = proto.CALL_COMMAND_NAME +\
   1350     self.command_header +\
   1351     args_command 